In [29]:
import numpy as np
import copy

# Discussion of Theory

To illustrate the Simplex algorithm, consider the following LP.

> Maximize $z = 4x_1 + 2x_2 + x_3$
>
> s.t.
>
> $x_1 \leq 1$
> 
> $4x_1 + x_2 + 6x_3 \leq 6$
>
> $8x_1 + 4x_2 + x_3 \leq 36$
>
> $x_1, x_2, x_3 \geq 0$

### Step 0: Reformat into Standard Form.
Remember that "Standard Form" for an LP is the following.

Max $z = \vec{c}^T \vec{x}$

s.t.

$\vec{\vec{A}} \vec{x} = \vec{b}$

$\vec{x} \geq 0$

We can reformat the original problem into this form using slack variables $s_1$, $s_2$, & $s_3$

> Maximize $z = 4x_1 + 2x_2 + x_3$
>
> s.t.
>
> $x_1 + s_1 = 1$
> 
> $4x_1 + x_2 + 6x_3 + s_3 = 6$
>
> $8x_1 + 4x_2 + x_3 + s_4 = 36$
>
> $x_1, x_2, x_3, s_1, s_2, s_3 \geq 0$

This standard matrix and vectors for this reformatted problem are as follows.

> $\vec{c}^T = \begin{pmatrix}
4 & 2 & 1 & 0 & 0 & 0\\
\end{pmatrix}$
>
> $\vec{\vec{A}} = \begin{pmatrix}
1 & 0 & 0 & 1 & 0 & 0\\
4 & 1 & 6 & 0 & 1 & 0\\
8 & 4 & 1 & 0 & 0 & 1
\end{pmatrix}$
>
> $\vec{x}^T = \begin{pmatrix}
x_1 & x_2 & x_3 & s_1 & s_2 & s_3\\
\end{pmatrix}$
>
> $\vec{b}^T = \begin{pmatrix}
1 & 6 & 36\\
\end{pmatrix}$

### Step 1: Guess a Basis
Now we can begin the actual Simplex algorithm.

Remember that the gist of the simplex algorithm is that an optimal solution to an LP will lie at a vertex of two or more constraints. The trick is to find which vertex is the optimal solution. For any randomly selected vertex we could check if we have the optimal solution by checking each neighboring vertex and comparing objective function values. If we found a neighboring vertex with a more optimal objective function value, then we try again on that vertex. If, by chance we picked the optimal vertex right off the bat, we could prove that we have the optimal solution relatively quickly. But if we already knew the optimal solution, we wouldn't need the Simplex algorithm in the first place. So, instead we select a random vertex and iterate from there until we find the optimal vertex.

Mathematically, we define a vertex as a series of active inequality constraints. In standard form, an active inequality constraint simply means that a respective variable is set to zero. In other words, selecting the appropriate active constraints is mathematically equivalent to setting appropriate variables to zero and solving the linear equation to find the values of the remaining variables. We call the remaining variables a "Basis" and the zero-values variables "Basic Variables".

In matrix form, it looks like this.
$\vec{\vec{A}} = \begin{pmatrix}
\vec{\vec{B}}\  \vert \vec{\vec{N}}\\
\end{pmatrix}$

$\vec{\vec{A}} \vec{x} = \vec{\vec{B}} x_B + \vec{\vec{N}} x_N = \vec{b}$

Where $\vec{\vec{B}}$ is an $m \times m$ matrix of the parts of $\vec{\vec{A}}$ pertaining the the non-basic varibles ($x_B$). Likewise $\vec{\vec{N}}$ is an $m \times (n-m)$ matrix of the parts of $\vec{\vec{A}}$ pertaining the the basic varibles ($x_N$). Where $m$ is the number of equality constraints and $n$ is the number of variables in the standard form problem.

Similarly, $\vec{c}$ can be split into appropriate sub vectors $\vec{c}_B$ and $\vec{c}_N$ containing the coefficients of $\vec{c}$ for the non-basic and basic variables respectively.

In our example, I'll guess that $x_1$, $x_2$, & $x_3$ are the basic variables. The resulting matrices and vectors are as follows.

> $\vec{\vec{B}} = \begin{pmatrix}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1
\end{pmatrix}$
>
> $\vec{\vec{N}} = \begin{pmatrix}
1 & 0 & 0\\
4 & 1 & 6\\
8 & 4 & 1
\end{pmatrix}$
>
> $\vec{x}_B^T = \begin{pmatrix}
s_1 & s_2 & s_3
\end{pmatrix}$
>
> $\vec{x}_N^T = \begin{pmatrix}
x_1 & x_2 & x_3
\end{pmatrix}$
>
> $\vec{c}_B^T = \begin{pmatrix}
0 & 0 & 0
\end{pmatrix}$
>
> $\vec{c}_N^T = \begin{pmatrix}
4 & 2 & 1
\end{pmatrix}$

### Step 2: Evaluate Optimality

Remember that, in order for the solution to be optimal, each neighboring vertex must have a worse objective function vaule than the chosen vertex. In other words, the objective function "cost" for switching from one vertex to each of it's neighbors must be positive. Fortunatley, the cost of switching vertices is easy to calculte. The formula for this "cost" (typically called reduced cost) is as follows.
$$RC = -\vec{c}_B^T \ \vec{\vec{B}}^{-1} \ \vec{\vec{N}}  + \vec{c}_N^T \leq \vec{0}^T$$

If this condition is met, then you've proven that the current vertex, with it's accompanying solution $\left( \vec{x}_B = \vec{\vec{B}}^{-1}\vec{b} \ , \ \vec{x}_N = \vec{0} \right)$ is optimal.

In the case of our example, the reduced cost vector is as follows.
> $RC = \begin{pmatrix}
4 & 2 & 1\\
\end{pmatrix} \nleq \vec{0}^T$

So we know our current solution is not optimal.

### Step 3: Determine how to swap variables into and out of the basis

Technically, you could pick any algorithm to swap variables into and out of the basis: So long as the condition in step 2 is satisfied, you can arrive at the optimal solution. But randomly moving variables around is not likely to get you to the optimal solution in a reasonable amount of time. Here, I'll present some common heuristics (rules of thumb) for determining which variables to move in and out. Profesional solvers are exceptionally good at knowing which variables to move in and out; that is why they're so fast.

#### Step 3-a: Select which variable to move from B to N
Since we want our reduced costs to be less than or equal to zero, a good choice to eliminate is the variable whose associated reduced cost is the highest.

In our example, the highest reduced cost value is $4$ which corresponds to $s_1$ in $\vec{x}_B$. So we'll select $s_1$ to move from B to N.

#### Step 3-b: Select which variable to move from N to B
Determining which incoming variable will produce the lowest reduced cost is a little more tricky and mathematically involved. But the gist is that you want to select the variable $i$ with the lowest positive value of $\frac{\left( \vec{\vec{B}}^{-1} \vec{b} \right)_i}{\left( \vec{\vec{B}}^{-1} \vec{\vec{N}} \right)_{i,j}}$ where $j$ is the variable selected in step 3-a.

In our example, $x_1$ produced this lowest positive value. So ultimately we will swap the matrix and vector coefficients of $s_1$ and $x_1$.

### Step 4: Repeat
One the appropriate swaps determined in step 3 and completed and the new matrices and vectors are assembled accordingly, return to step 2 and repeat.

# Python Implementation

In [46]:
# From the description above (see the end of Step 1)

B = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1]
])
N = np.array([
    [1,0,0],
    [4,1,6],
    [8,4,1]
])

xB = np.array(["s1","s2","s3"])
xN = np.array(["x1","x2","x3"])

cB = np.array([
    [0,],
    [0,],
    [0,]
])
cN = np.array([
    [4,],
    [2,],
    [1,]
])

b = np.array([
    [1,],
    [6,],
    [36,]
])

def computeRC(cB,B,N,cN):
    return -np.matmul(np.matmul(np.transpose(cB),np.linalg.inv(B)),N) + np.transpose(cN)

def isOptimal(RC):
    for n in np.squeeze(RC): #squeeze simply reduces the matrix to vector
        if n > 0:
            return False
    return True

RC = computeRC(cB,B,N,cN)

while not isOptimal(RC):
    #Determine variables to swap
    BtoNIndex = np.argmax(RC)
    
    Binv = np.linalg.inv(B) #Note that this is a computationally expensive operation.
    Binvb = np.matmul(Binv,b)
    BinvN = np.matmul(Binv,N)
    
    testArray = np.array([Binvb[i] / BinvN[i,BtoNIndex] for i in range(len(B))])
    NtoBIndex = np.where(testArray > 0, testArray, np.inf).argmin()
    
    #SwapVariables
    temp = copy.deepcopy(cB[BtoNIndex,0])
    cB[BtoNIndex,0] = cN[NtoBIndex,0]
    cN[NtoBIndex,0] = temp
    
    temp = copy.deepcopy(xB[BtoNIndex])
    xB[BtoNIndex] = xN[NtoBIndex]
    xN[NtoBIndex] = temp
    
    temp = copy.deepcopy(B[:,BtoNIndex])
    B[:,BtoNIndex] = N[:,NtoBIndex]
    N[:,NtoBIndex] = temp
    
    RC = computeRC(cB,B,N,cN)
    
#Compile the results
xBValues = np.matmul(np.linalg.inv(B),b)

allVars = {
    "x1": None,
    "x2": None,
    "x3": None,
    "s1": None,
    "s2": None,
    "s3": None
}

for i in range(len(xB)):
    varName = xB[i]
    value = xBValues[i,0]
    allVars[varName] = value 
    
for i in range(len(xN)):
    varName = xN[i]
    value = 0.0
    allVars[varName] = value 
    
displayVars = ["x1","x2","x3"]
    
for var in displayVars:
    print("{}: {}".format(var,allVars[var]))

x1: 0.0
x2: 6.0
x3: 0.0


C:\Users\nb5786-A\AppData\Local\Temp\ipykernel_21872\1872443589.py:53: RuntimeWarning: divide by zero encountered in divide
  testArray = np.array([Binvb[i] / BinvN[i,BtoNIndex] for i in range(len(B))])
